In [6]:
#
# Imports and general settings
#
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337);

In [8]:
#
# Prepare training data
#

# read in file
with open('../TrainingData/shakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(list(set(text)))
vocabulary_size = len(chars)

# Build encoder and decoder
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

# create data tensors
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
training_data = data[:n]
validation_data = data[n:]

In [18]:
#
# Hyper parameter
#

block_size = 8 # number of consecutive characters to predict from
batch_size = 32 # mini-batch size

In [10]:
#
# mini-batch creation
#
def get_batch(split):
    data = training_data if split == 'train' else validation_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

In [16]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, vocab_size)
    def forward(self, idx, targets=None):
        logits = self.token_embedding(idx)
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    def generate(self, idx, max_tokens):
        for _ in range(max_tokens):
            logits, _ = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx


In [17]:
m = BigramLanguageModel(vocabulary_size)



jeqpjtESpJLga-eqqzwnQaKYdhgjnlS.3Dga-QPqGWl!W$CkIb;zwvfOobTvJZKL;rn3;q$UT3d'oH
bYQqbOTfV! g;yJ-r;tNb


In [22]:
optimizer = torch.optim.AdamW(m.parameters(), lr=0.1)
for steps in range(10000):
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

2.5947859287261963


In [24]:
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_tokens=500)[0].tolist()))


Counow sttharls, t in leyst ms tha nleincede jay,
Dours othe s key wepie

KEAno--I noldgrmig pou:
Co nlllingin havenaveresus, t tow:
Withen my ld weremereaing blturiat, otimunt hiny ille, athin nd, INTI thonrchemo th.
Dies my al hierd ep ha s?
ARS:
Semes thinghy. IORDitwint wathamine actwis 's t s.
VI, de vir XENo th inous whund y
I wagy te
Fisoore hemy; s? are tll'lllincar:
Difous r gow blul greit d ace bavelay al!
I ouph:
Theed o th
Wi'sll, atoye:
Therd unes ico o-f, coss
NRI seean ay ud a mem
